In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor


pd.set_option("display.max_columns", None)

# Dataset credit goes to
#### https://github.com/LuisM78/Appliances-energy-prediction-data
#### https://archive.ics.uci.edu/dataset/374/appliances+energy+prediction

##### The data set is at 10 min for about 4.5 months. The house temperature and humidity conditions were monitored with a ZigBee wireless sensor network. Each wireless node transmitted the temperature and humidity conditions around 3.3 min. Then, the wireless data was averaged for 10 minutes periods. The energy data was logged every 10 minutes with m-bus energy meters. Weather from the nearest airport weather station (Chievres Airport, Belgium) was downloaded from a public data set from Reliable Prognosis (rp5.ru), and merged together with the experimental data sets using the date and time column. Two random variables have been included in the data set for testing the regression models and to filter out non predictive attributes (parameters).

| Variable Name | Role     | Type        | Description | Units  | Missing Values |
|----------------|----------|-------------|--------------|--------|----------------|
| date           | Feature  | Date        | Timestamp of observation | — | no |
| Appliances     | Target   | Integer     | Energy use in household appliances | Wh | no |
| lights         | Feature  | Integer     | Energy use of lights in the house | Wh | no |
| T1             | Feature  | Continuous  | Temperature in kitchen area | °C | no |
| RH_1           | Feature  | Continuous  | Humidity in kitchen area | % | no |
| T2             | Feature  | Continuous  | Temperature in living room | °C | no |
| RH_2           | Feature  | Continuous  | Humidity in living room | % | no |
| T3             | Feature  | Continuous  | Temperature in laundry room | °C | no |
| RH_3           | Feature  | Continuous  | Humidity in laundry room | % | no |
| T4             | Feature  | Continuous  | Temperature in office room | °C | no |
| RH_4           | Feature  | Continuous  | Humidity in office room | % | no |
| T5             | Feature  | Continuous  | Temperature in bathroom | °C | no |
| RH_5           | Feature  | Continuous  | Humidity in bathroom | % | no |
| T6             | Feature  | Continuous  | Temperature in north bedroom | °C | no |
| RH_6           | Feature  | Continuous  | Humidity in north bedroom | % | no |
| T7             | Feature  | Continuous  | Temperature in ironing room | °C | no |
| RH_7           | Feature  | Continuous  | Humidity in ironing room | % | no |
| T8             | Feature  | Continuous  | Temperature in teenager room | °C | no |
| RH_8           | Feature  | Continuous  | Humidity in teenager room | % | no |
| T9             | Feature  | Continuous  | Temperature in parents’ room | °C | no |
| RH_9           | Feature  | Continuous  | Humidity in parents’ room | % | no |
| T_out          | Feature  | Continuous  | Outdoor temperature | °C | no |
| Press_mm_hg    | Feature  | Continuous  | Outdoor pressure | mm Hg | no |
| RH_out         | Feature  | Continuous  | Outdoor humidity | % | no |
| Windspeed      | Feature  | Continuous  | Wind speed | m/s | no |
| Visibility     | Feature  | Continuous  | Outdoor visibility | km | no |
| Tdewpoint      | Feature  | Continuous  | Dew point temperature | °C | no |
| rv1            | Feature  | Continuous  | Random variable 1 (for testing purposes) | — | no |
| rv2            | Feature  | Continuous  | Random variable 2 (for testing purposes) | — | no |



# Load the Dataset

In [4]:
CSV_LOCATION = "Dataset/energydata_complete.csv"

df = pd.read_csv(CSV_LOCATION)
df.sample(2)

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
15278,2016-04-26 19:20:00,130,0,21.760000,38.826667,20.166667,39.066667,21.856667,36.090000,20.390000,...,19.390000,32.626667,5.966667,749.000000,63.333333,6.333333,40.0,-0.6,21.002241,21.002241
18961,2016-05-22 09:10:00,120,0,24.458182,53.041364,23.456757,52.514775,27.348636,46.668182,23.767568,...,23.264091,50.718182,15.416667,750.266667,93.000000,3.166667,40.0,14.3,0.834888,0.834888


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19735 entries, 0 to 19734
Data columns (total 29 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   date         19735 non-null  object 
 1   Appliances   19735 non-null  int64  
 2   lights       19735 non-null  int64  
 3   T1           19735 non-null  float64
 4   RH_1         19735 non-null  float64
 5   T2           19735 non-null  float64
 6   RH_2         19735 non-null  float64
 7   T3           19735 non-null  float64
 8   RH_3         19735 non-null  float64
 9   T4           19735 non-null  float64
 10  RH_4         19735 non-null  float64
 11  T5           19735 non-null  float64
 12  RH_5         19735 non-null  float64
 13  T6           19735 non-null  float64
 14  RH_6         19735 non-null  float64
 15  T7           19735 non-null  float64
 16  RH_7         19735 non-null  float64
 17  T8           19735 non-null  float64
 18  RH_8         19735 non-null  float64
 19  T9  

In [6]:
df.dtypes

date            object
Appliances       int64
lights           int64
T1             float64
RH_1           float64
T2             float64
RH_2           float64
T3             float64
RH_3           float64
T4             float64
RH_4           float64
T5             float64
RH_5           float64
T6             float64
RH_6           float64
T7             float64
RH_7           float64
T8             float64
RH_8           float64
T9             float64
RH_9           float64
T_out          float64
Press_mm_hg    float64
RH_out         float64
Windspeed      float64
Visibility     float64
Tdewpoint      float64
rv1            float64
rv2            float64
dtype: object

In [10]:
df.describe()

,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,RH_5,T6,RH_6,T7,RH_7,T8,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
count,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000
mean,97.694958,3.801875,21.686571,40.259739,20.341219,40.420420,22.267611,39.242500,20.855335,39.026904,19.592106,50.949283,7.910939,54.609083,20.267106,35.388200,22.029107,42.936165,19.485828,41.552401,7.411665,755.522602,79.750418,4.039752,38.330834,3.760707,24.988033,24.988033
std,102.524891,7.935988,1.606066,3.979299,2.192974,4.069813,2.006111,3.254576,2.042884,4.341321,1.844623,9.022034,6.090347,31.149806,2.109993,5.114208,1.956162,5.224361,2.014712,4.151497,5.317409,7.399441,14.901088,2.451221,11.794719,4.194648,14.496634,14.496634
min,10.000000,0.000000,16.790000,27.023333,16.100000,20.463333,17.200000,28.766667,15.100000,27.660000,15.330000,29.815000,-6.065000,1.000000,15.390000,23.200000,16.306667,29.600000,14.890000,29.166667,-5.000000,729.300000,24.000000,0.000000,1.000000,-6.600000,0.005322,0.005322
25%,50.000000,0.000000,20.760000,37.333333,18.790000,37.900000,20.790000,36.900000,19.530000,35.530000,18.277500,45.400000,3.626667,30.025000,18.700000,31.500000,20.790000,39.066667,18.000000,38.500000,3.666667,750.933333,70.333333,2.000000,29.000000,0.900000,12.497889,12.497889
50%,60.000000,0.000000,21.600000,39.656667,20.000000,40.500000,22.100000,38.530000,20.666667,38.400000,19.390000,49.090000,7.300000,55.290000,20.033333,34.863333,22.100000,42.375000,19.390000,40.900000,6.916667,756.100000,83.666667,3.666667,40.000000,3.433333,24.897653,24.897653
75%,100.000000,0.000000,22.600000,43.066667,21.500000,43.260000,23.290000,41.760000,22.100000,42.156667,20.619643,53.663333,11.256000,83.226667,21.600000,39.000000,23.390000,46.536000,20.600000,44.338095,10.408333,760.933333,91.666667,5.500000,40.000000,6.566667,37.583769,37.583769
max,1080.000000,70.000000,26.260000,63.360000,29.856667,56.026667,29.236000,50.163333,26.200000,51.090000,25.795000,96.321667,28.290000,99.900000,26.000000,51.400000,27.230000,58.780000,24.500000,53.326667,26.100000,772.300000,100.000000,14.000000,66.000000,15.500000,49.996530,49.996530


In [12]:
# Datetime features

df["date"] = pd.to_datetime(df["date"])
df["hour"] = df["date"].dt.hour
df["day_of_week"] = df["date"].dt.dayofweek
df["month"] = df["date"].dt.month
df["is_weekend"] = df["day_of_week"].isin([5, 6]).astype(int)

In [13]:
df.sample(2)

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,RH_5,T6,RH_6,T7,RH_7,T8,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2,hour,day_of_week,month,is_weekend
17971,2016-05-15 12:10:00,100,0,23.175,35.4975,23.100000,33.700000,24.00,34.567143,23.7,34.030000,23.2,37.512857,14.063333,1.0,23.60,28.390,25.26,35.23,23.0,33.652000,10.716667,762.1,57.500000,5.0,40.000000,2.683333,5.713364,5.713364,12,6,5,1
19703,2016-05-27 12:50:00,70,0,24.890,47.7900,26.571429,40.384286,28.79,42.430000,24.5,44.826667,22.7,52.863333,24.593333,1.0,23.89,43.312,23.79,48.90,23.1,46.466667,20.916667,756.1,61.166667,1.0,30.833333,13.116667,28.464212,28.464212,12,4,5,0
